This notebook demonstrates the different functionality and capabilities of the Knowledege-consistent hybrid AI networks (K-CHAIN). Specifically, the demonstration will cover the following aspects:

-  Setup of K-CHAIN
-  Building models:
    -  with physics models with simple equations provided as strings
    -  with physics models captured in TF-compatible python code, where the code itself was derived from extracted text
    -  with experimental data



In [1]:
import tensorflow as tf
from tensorflow.contrib import autograph as ag
import requests

In [3]:
def runSessionWithDefaults(sess, mdl, output, fd, defaultValues, defaultValuesUsed):
    try:
        aval = sess.run(output, feed_dict=fd)
    except tf.errors.InvalidArgumentError as e:
        #look for value available in default values
        if e.node_def.name in defaultValues.keys():
            tmp = mdl.get_tensor_by_name(e.node_def.name+':0')
            fd[tmp] = defaultValues[e.node_def.name]

            defaultValuesUsed[e.node_def.name] = defaultValues[e.node_def.name]

            aval, defaultValuesUsed = runSessionWithDefaults(sess, mdl, output, fd, defaultValues, defaultValuesUsed)
        else:
            print('Please provide value for : ' + e.node_def.name)
            aval = None
    return aval, defaultValuesUsed

In [4]:
def speedOfSound(invars):
    T = invars[0]
    R = invars[1]
    gamma = invars[2]
    a = tf.sqrt(R*gamma*T, name=outputVar[0]['name'])
    return a

# Create a clean graph
mdl = tf.Graph()
in_dims = len(inputVar)
with mdl.as_default():
    invars = []
    for node in inputVar:
        tfType = tf.float32
        invars.append(tf.placeholder(tfType, name=node['name']))
    
    tfType = tf.float32
    tf_model = ag.to_graph(speedOfSound)
    output = tf_model(invars)
    tf.add_to_collection("output", output)

    for node in invars:
        tf.add_to_collection("input", node)  


evalInputVar=[
    {
        "name": "T",
        "type": "float",
        "value": 300.0
    },
    {
        "name": "R",
        "type": "float",
        "value": 300.0
    },
    {
        "name": "gamma",
        "type": "float",
        "value": 1.40
    }
]
sess = tf.Session(graph = mdl)

fd = {}
for node in evalInputVar:
    fd[mdl.get_tensor_by_name(node['name']+':0')]=node['value']

defaultValuesUsed = {}
aval, defaultValuesUsed = runSessionWithDefaults(sess, mdl, output, fd, defaultValues, defaultValuesUsed)   

print('output: '+str(aval))
print('default values used: '+str(defaultValuesUsed))

sess.close()

output: 354.96478
default values used: {}


In [10]:
url_build = 'http://localhost:12345/darpa/aske/kchain/build'
url_evaluate = 'http://localhost:12345/darpa/aske/kchain/evaluate'

In [11]:
inputPacket = {
                  "inputVariables": [
                    {
                      "name": "Mass",
                      "type": "double"
                    },
                    {
                      "name": "Acceleration",
                      "type": "double"
                    }
                  ],
                  "outputVariables": [
                    {
                      "name": "Force",
                      "type": "double"
                    }
                  ],
                   "equationModel" : "Force = Mass * Acceleration",
                   "modelName" : "Newtons2LawModel"
                 }
r = requests.post(url_build, json=inputPacket)
r.json()

{'metagraphLocation': '../models/Newtons2LawModel',
 'modelType': 'Physics',
 'trainedState': 0}

{"inputVars": ["R", "T", "gamma", "theta"], "originalLHS": "a**2", "originalRHS": " R * T *  (  1 * ( gamma-1 ) / ( 1 * ( gamma-1 ) *  (  ( theta/T ) ** 2 *  tf.math.exp( theta/T ) / (  tf.math.exp( theta/T ) - 1 ) ** 2 )  ) ) ", "modifiedLHS": "a", "modifiedRHS": "tf.math.pow( R * T *  (  1 * ( gamma-1 ) / ( 1 * ( gamma-1 ) *  (  ( theta/T ) ** 2 *  tf.math.exp( theta/T ) / (  tf.math.exp( theta/T ) - 1 ) ** 2 )  ) ) , 1/2)", "fullEquation": "a = tf.math.pow( R * T *  (  1 * ( gamma-1 ) / ( 1 * ( gamma-1 ) *  (  ( theta/T ) ** 2 *  tf.math.exp( theta/T ) / (  tf.math.exp( theta/T ) - 1 ) ** 2 )  ) ) , 1/2)", "TF operations": ["tf.math.pow", "tf.math.exp"]}

In [18]:
inputPacket = {
                  "inputVariables": [
                    {
                        "name": "gamma",
                        "type": "double",
                        "value": "1.4"
                    },
                    {
                        "name": "R",
                        "type": "double",
                        "value": "286.0"
                    },
                    {
                        "name": "theta",
                        "type": "double",
                        "value": "3056.0"
                    },
                    {
                      "name": "T",
                      "type": "double"
                    }
                  ],
                  "outputVariables": [
                    {
                      "name": "a",
                      "type": "double"
                    }
                  ],
                   "equationModel" : "a = tf.math.pow( R * T *  (  1 + ( gamma-1 ) / ( 1 + ( gamma-1 ) *  (  ( theta/T ) ** 2 *  tf.math.exp( theta/T ) / (  tf.math.exp( theta/T ) - 1 ) ** 2 )  ) ) , 1/2)",
                   "modelName" : "SpeedOfSound"
                 }
r = requests.post(url_build, json=inputPacket)
r.json()

{'metagraphLocation': '../models/SpeedOfSound',
 'modelType': 'Physics',
 'trainedState': 0}

In [28]:
evalPacket = {
  "inputVariables": [
    {
      "name": "T",
      "type": "double",
      "value": "[300.0, 273.00]"
    }
  ],
  "modelName": "SpeedOfSound",
  "outputVariables": [
    {
      "name": "a",
      "type": "double"
    }
  ]
}
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'defaultsUsed': [{'name': 'theta', 'value': '3056.0'},
  {'name': 'R', 'value': '286.0'},
  {'name': 'gamma', 'value': '1.4'}],
 'outputVariables': [{'name': 'a',
   'type': 'double',
   'value': '[346.50601552,330.58687602]'}]}